#4. 단어사전만들기
- 1. 전체 단어 사전만들기
- 2. 2만단어 사전 만들기

In [ ]:
# 코랩에서 실행시 pyMySQL을 먼저 설치해야 한다
!pip install JPype1
# !pip install konlpy
!pip install pyMySQL

In [ ]:
# import packages
import os
import numpy as np
import pandas as pd
#import FinanceDataReader as fdr
#import konlpy
#from konlpy.tag import Okt
import pymysql
from os import replace
import requests
from tqdm.notebook import tqdm
from datetime import datetime, timedelta
from sqlalchemy import create_engine
import ast
from os import replace
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
import re
import string
import itertools
import warnings
warnings.filterwarnings('ignore')

---
1. 전체 단어 사전 만들기
---

In [ ]:
# get_oneday_sokbo_from_db() -------------------------------------------
# 지정한 날짜의 하루치 뉴스를 DB에서 읽어오기
def get_oneday_sokbo_from_db(sdate):
  # DB 연결 준비: 뉴스(2줄 속보) 읽어오기
  conn = pymysql.connect(host= '호스트주소', port = 포트번호, user="아이디", password="비밀번호", db="DB이름", charset = 'utf8')
  # DB에서 뉴스(summary, press, rdate) 추출위한 sql 준비
  sql = f"SELECT x.* FROM DB이름.{sdate}_sokbo x"
  # DB 검색결과를 dataframe에 저장
  result_df = pd.read_sql_query(sql, conn)
  # 실행확인을 위한 화면 출력
  print(sdate)
  # DB close
  conn.close()
  # 결과 반환
  return result_df
# get_oneday_sokbo_from_db(sdate) :END---------------------------------------

# get_sokbo_from_db() -------------------------------------------------------
# 주어진 기간의 뉴스 읽어오기
def get_sokbo_from_db(startDate,endDate):
  # 시작날짜 ddate 복사
  ddate = startDate
  # 종료 날짜까지 반복하기
  result_df = pd.DataFrame()
  while pd.to_datetime(ddate).date() <= pd.to_datetime(endDate).date():
    # 해당일 하루 뉴스 가져오기
    temp_df = get_oneday_sokbo_from_db(ddate)
    # 기존 dataframe에 추가 하기
    result_df = pd.concat([result_df,temp_df],axis=0,ignore_index=True)
    # 자동 날짜 증가 연산: 1일씩 증가
    dt_date = pd.to_datetime(ddate).date() + timedelta(days=1)
    # 다음 날짜 문자열 추출
    ddate= dt_date.strftime("%Y%m%d")
  return result_df
# -- get_sokbo_from_db() :END------------------------------------------------

In [ ]:
############################################################################
# 전체 단어사전 만들기: TextVectorization 커스터마이즈
############################################################################
# TextVectorization 을 customize해보기
# 표준화, 토큰화 함수를 정의해서 지정하기
# 리스트 flatten을 위한 itertools 패키지 import

# -- custom_standardization_fn() --------------------------------------------
# custom_표준화 함수
def custom_standardization_fn(string_tensor):
    lowercase_string = string_tensor # 영문의 경우 여기서 모두 소문자 처리, 한글은 필요없음
    return tf.strings.regex_replace(
        # 문장부호 처리: %와 .는 살리고 나머지는 제외
        lowercase_string, f"[{re.escape(string.punctuation.replace('%','').replace('.',''))}]", "") 
# -- custom_standardization_fn() :END----------------------------------------

# -- custom_split_fn()-------------------------------------------------------
# custom_토큰화 함수
def custom_split_fn(string_tensor):
    return tf.strings.split(string_tensor)  # 한글의 경우 형태소 분석 처리 해야 한다
# -- custom_split_fn() :END--------------------------------------------------
 

# custom 표준화, 토큰화 함수 지정해서 TextVectoriztion 생성하기 ---------------
text_vectorization = TextVectorization(
    # 정수 인덱스로 출력
    output_mode="int",
    # 표준화 함수를 custom_표준화 함수로 지정
    standardize=custom_standardization_fn,
    # 토큰화 함수를 custom_토큰화 함수로 지정
    split=custom_split_fn,
)
# custom 표준화, 토큰화 함수 지정해서 TextVectoriztion 생성하기 END------------

In [ ]:
# --네이버증권실시간속보 2줄뉴스 text로 전체 단어사전만들기------------------------------
# db로 부터 데이터를 읽어오기
df_test = get_sokbo_from_db('20230101','20240202')
# 사전만들기 위한 토큰화된 text를 넣은 dataset 리스트 생성
dataset = []
# 전체 data 반복
for i in range (len(df_test)):
  # 토큰화된 text 를 하나의 리스트로 만든다
  dataset.append(eval(df_test['tokenized_summary'][i]))
#  중첩된 리스트를 flatten 시킨다
dataset = list(itertools.chain(*dataset))

# ============================================
# .adapt()로 단어 사전 만들기
text_vectorization.adapt(dataset)
# ============================================

# 어휘사전 확인 .get_vocabulary() 함수
text_vectorization.get_vocabulary()

# -- 네이버증권실시간속보 2줄뉴스 text로 사전만들기 :END ----------------------

---
- 전체 단어사전 pickle로 저장
---

In [ ]:
# =======================================================
# TextVectorizer 저장, 사전 vocabulary 저장하기
# --- 저장 : start ---
# Vector for word "this"
print (text_vectorization("LG"))

# Pickle the config and weights
pickle.dump({'config': text_vectorization.get_config(),
             'weights': text_vectorization.get_weights()}
            , open("tv_layer.pkl", "wb"))

# Later you can unpickle and use
# `config` to create object and
# `weights` to load the trained weights.
# --- 저장 : end ---
# =========================================================

---
- pickle로 저장된 전체 단어사전 load 하기
---

In [ ]:
# =========================================================
# 저장된 TextVectorizer 사전 vocabulary load 하기
# --- 불러오기 load : start ---
from_disk = pickle.load(open("tv_layer.pkl", "rb"))

new_vectorizer = TextVectorization(
    # 정수 인덱스로 출력
    output_mode="int",
    # 표준화 함수를 custom_표준화 함수로 지정
    standardize=custom_standardization_fn,
    # 토큰화 함수를 custom_토큰화 함수로 지정
    split=custom_split_fn,
  )
new_vectorizer.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
new_vectorizer.set_weights(from_disk['weights'])

# Lets see the Vector for word "this"
print (new_vectorizer("LG"))
# --- 불러오기 load : end ---
# =============================================================

---
2. 2만 단어 사전 만들기
- max_tokens 지정: max_tokens=20000
---

In [ ]:
##########################################################
# 2만단어 단어사전 만들기: TextVectorization 커스터마이즈
##########################################################
from tensorflow.keras.layers import TextVectorization
# TextVectorization 을 customize해보기
# 표준화, 토큰화 함수를 정의해서 지정하기
# -- custom_standardization_fn() --------------------------------------------
# custom_표준화 함수
def custom_standardization_fn(string_tensor):
    lowercase_string = string_tensor # 영문의 경우 여기서 모두 소문자 처리, 한글은 필요없음
    return tf.strings.regex_replace(
        # 문장부호 처리: %와 .는 살리고 나머지는 제외
        lowercase_string, f"[{re.escape(string.punctuation.replace('%','').replace('.',''))}]", "") 
# -- custom_standardization_fn() :END----------------------------------------

# -- custom_split_fn()-------------------------------------------------------
# custom_토큰화 함수
def custom_split_fn(string_tensor):
    return tf.strings.split(string_tensor)  # 한글의 경우 형태소 분석 처리 해야 한다
# -- custom_split_fn() :END--------------------------------------------------

# custom 표준화, 토큰화 함수 지정해서 TextVectoriztion 생성하기 ---------------
text_vectorization = TextVectorization(
    #--- max_tokens 지정--
    max_tokens=20000
    #---------------------
    # 정수 인덱스로 출력
  ,  output_mode="int"
    # 표준화 함수를 custom_표준화 함수로 지정
  ,  standardize=custom_standardization_fn
    # 토큰화 함수를 custom_토큰화 함수로 지정
  ,  split=custom_split_fn
)
# custom 표준화, 토큰화 함수 지정해서 TextVectoriztion 생성하기 :END------------

In [ ]:
# --네이버증권실시간속보 2줄뉴스 text로 2만단어사전만들기-------------------------
# db로 부터 데이터를 읽어오기
df_test = get_sokbo_from_db('20230101','20240202')
# 사전만들기 위한 토큰화된 text를 넣은 dataset 리스트 생성
dataset = []
# 전체 data 반복
for i in range (len(df_test)):
  # 토큰화된 text 를 하나의 리스트로 만든다
  dataset.append(eval(df_test['tokenized_summary'][i]))
#  중첩된 리스트를 flatten 시킨다
dataset = list(itertools.chain(*dataset))

# ============================================
# .adapt()로 단어 사전 만들기
text_vectorization.adapt(dataset)
# ============================================

# 어휘사전 확인 .get_vocabulary() 함수
len(text_vectorization.get_vocabulary())
# --네이버증권실시간속보 2줄뉴스 text로 2만단어사전만들기 :END--------------------

---
- 2만 단어 사전 pickle로 저장
---

In [ ]:
# =======================================================
# TextVectorizer 저장, 2만단어 사전 vocabulary 저장하기
# --- 저장 : start ---
# Vector for word "this"
print (text_vectorization("LG"))

# Pickle the config and weights
pickle.dump({'config': text_vectorization.get_config(),
             'weights': text_vectorization.get_weights()}
            , open("tv_layer_20000.pkl", "wb"))

# Later you can unpickle and use
# `config` to create object and
# `weights` to load the trained weights.
# --- 저장 : end ---
# =========================================================

---
* pickle로 저장된 2만 단어 사전 load 하기
---

In [ ]:
# =========================================================
# 저장된 TextVectorizer 2만단어사전 vocabulary load 하기
# --- 불러오기 load : start ---
from_disk = pickle.load(open("tv_layer_20000.pkl", "rb"))

new_vectorizer = TextVectorization(
    # 정수 인덱스로 출력
    output_mode="int",
    # 표준화 함수를 custom_표준화 함수로 지정
    standardize=custom_standardization_fn,
    # 토큰화 함수를 custom_토큰화 함수로 지정
    split=custom_split_fn,
  )
new_vectorizer.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
new_vectorizer.set_weights(from_disk['weights'])

# Lets see the Vector for word "this"
print (new_vectorizer("LG"))
# --- 불러오기 load : end ---
# =============================================================